In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/removeinvalid/marinedata_train.csv
/kaggle/input/removeinvalid/marinedata_test.csv
/kaggle/input/removeinvalid/marinedata_val.csv


In [3]:
dataTrain = pd.read_csv("/kaggle/input/removeinvalid/marinedata_train.csv")
dataTest = pd.read_csv("/kaggle/input/removeinvalid/marinedata_test.csv")
dataVal = pd.read_csv("/kaggle/input/removeinvalid/marinedata_val.csv")

In [12]:
dataTrain.head()

,timestamp,mmsi,lat,lon,speed,heading,timePassed,timeStep,path,timePassedTotal,torque,alat,alon,aspeed,aheading,pathIni
0,2017-12-22 10:23:57,205366000,57.756818,10.420237,9.16,79.0,4316690,1973822,4,1513938237,0,0.340376,0.048352,0.1832,0.219444,0
1,2017-12-22 10:34:03,205366000,57.767407,10.509043,8.90,79.0,4317296,606,4,1513938843,0,0.341204,0.052561,0.1780,0.219444,0
2,2017-12-22 10:45:13,205366000,57.777053,10.594123,8.18,85.0,4317966,670,4,1513939513,0,0.341957,0.056594,0.1636,0.236111,0
3,2017-12-22 10:55:19,205366000,57.776770,10.678505,8.33,93.0,4318572,606,4,1513940119,0,0.341935,0.060593,0.1666,0.258333,0
4,2017-12-22 11:04:30,205366000,57.768455,10.734610,3.55,163.0,4319123,551,4,1513940670,0,0.341286,0.063252,0.0710,0.452778,0


In [25]:
import math
# Function to calculate distance/speed of ship between timestamps
def DistSpeed(latOld, lonOld, latNew, lonNew, timePassed):
    # first calculate distance
    p1 = (math.sin(math.radians(latOld)) * math.sin(math.radians(latNew))) + (math.cos(math.radians(latOld)) * math.cos(math.radians(latNew))) * (math.cos(math.radians(lonNew) - math.radians(lonOld)))
    # check needed because acos(1.0000002) is invalid
    if(p1 > 1):
        p1 = 1
    dist = math.acos(p1) * 6371
    # convert from km to nautical miles
    nauticalMiles = dist * 0.5399568035
    # divide distance by timePassed to get knots, formula: distance * 3600 / timePassed (since timePassed is in seconds)
    speed = nauticalMiles * 3600 / timePassed
    return speed

In [21]:
DistSpeed(57.756818, 10.420237, 57.767407, 10.509043, 606) / 1.852
# The speed in in km/hr 

9.34855624672426

In [48]:
data = dataVal

In [49]:
latOld = 0
lonOld = 0
pathOn = -1
data["Elim"] = 0.0

for i in range(len(data["pathIni"])):
    row = data.iloc[i]
    
    if(row["pathIni"] == pathOn):
        # Calculate speed
        speed = DistSpeed(latOld, lonOld, row["lat"], row["lon"], row["timeStep"])
        if(speed > 40):
            # speed > 40 knots, probably not a valid point
            data.iloc[i, data.columns.get_loc("Elim")] = speed
    else:
        # skip check step, update pathOn
        pathOn = row["pathIni"]
    # at end, set up latOld and lonOld
    latOld = row["lat"]
    lonOld = row["lon"]

In [50]:
badData = data[data["Elim"] != 0]
badData.head()

,timestamp,mmsi,lat,lon,speed,heading,timePassed,timeStep,path,timePassedTotal,torque,alat,alon,aspeed,aheading,pathIni,Elim
22372,2018-09-09 17:56:22,246571000,59.365112,23.888415,0.00,63.0,26988078,361,164972,1536515782,0,0.466024,0.686655,0.0000,0.175000,299,47.640819
22373,2018-09-09 18:03:28,246571000,59.354403,24.043128,0.00,150.0,26988504,426,164972,1536516208,0,0.465188,0.693987,0.0000,0.416667,299,40.373980
28068,2018-07-24 05:57:09,246625000,57.127883,23.864250,0.00,44.0,20861295,556,166443,1532411829,0,0.291241,0.685509,0.0000,0.122222,377,75.036282
28069,2018-07-24 06:11:55,246625000,56.982233,24.097150,0.00,4.0,20862181,886,166443,1532412715,0,0.279862,0.696547,0.0000,0.011111,377,47.088231
51002,2018-08-31 15:47:28,246845000,65.022308,25.332205,6.02,307.0,25288079,361,171530,1535730448,0,0.907993,0.755081,0.1204,0.852778,659,2753.502088


In [51]:
badData.shape

(19, 17)

In [52]:
goodData = data[data["Elim"] == 0]
goodData.head()
goodData.drop(columns=['Elim'])

,timestamp,mmsi,lat,lon,speed,heading,timePassed,timeStep,path,timePassedTotal,torque,alat,alon,aspeed,aheading,pathIni
0,2017-12-07 01:47:10,246403000,54.577517,12.474700,5.25,61.0,2366896,571,160672,1512611230,0,0.091993,0.145720,0.1050,0.169444,0
1,2017-12-07 02:03:49,246403000,54.598717,12.541650,5.30,61.0,2367895,999,160672,1512612229,0,0.093650,0.148893,0.1060,0.169444,0
2,2017-12-07 02:14:19,246403000,54.613117,12.587817,5.50,60.0,2368525,630,160672,1512612859,0,0.094775,0.151081,0.1100,0.166667,0
3,2017-12-07 02:23:48,246403000,54.626750,12.634117,5.35,64.0,2369094,569,160672,1512613428,0,0.095840,0.153276,0.1070,0.177778,0
4,2017-12-07 02:34:48,246403000,54.638483,12.678450,5.25,64.0,2369754,660,160672,1512614088,0,0.096757,0.155377,0.1050,0.177778,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108612,2018-09-17 18:03:57,249500000,60.033937,19.351617,7.87,323.0,27679903,600,182826,1537207437,0,0.518276,0.471641,0.1574,0.897222,1423
108613,2018-09-17 18:14:03,249500000,60.068167,19.299523,7.92,323.0,27680509,606,182826,1537208043,0,0.520951,0.469172,0.1584,0.897222,1423
108614,2018-09-17 18:24:09,249500000,60.102227,19.247868,7.87,323.0,27681115,606,182826,1537208649,0,0.523611,0.466724,0.1574,0.897222,1423
108615,2018-09-17 18:33:51,249500000,60.135040,19.198073,7.87,323.0,27681697,582,182826,1537209231,0,0.526175,0.464364,0.1574,0.897222,1423


In [53]:
goodData.to_csv("marinedata_val.csv", index=False)